In [1]:
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import h5py
import nbimporter
from Functions import process_file, process_file_optimized

In [20]:
labs = pd.read_csv('D:/PulseDB_Vital_labs.csv')
labs.head()

,caseid,dt,name,result
0,1,594470,alb,2.9
1,1,399575,alb,3.2
2,1,12614,alb,3.4
3,1,137855,alb,3.6
4,1,399575,alt,12.0


In [21]:
#pad the caseid with p and 6 zeros
labs['caseid'] = labs['caseid'].apply(lambda x: 'p' + str(x).zfill(6))

#Transform dt to datetime
labs['dt'] = pd.to_datetime(labs['dt'], unit='s')

labs.head()

,caseid,dt,name,result
0,p000001,1970-01-07 21:07:50,alb,2.9
1,p000001,1970-01-05 14:59:35,alb,3.2
2,p000001,1970-01-01 03:30:14,alb,3.4
3,p000001,1970-01-02 14:17:35,alb,3.6
4,p000001,1970-01-05 14:59:35,alt,12.0


In [22]:
legend = pd.read_excel('D:/simedy/CSVs/PulseDBVital_legend.xlsx')
legend.head()

,Parameter,Data Source,Category,Description,Unit,Reference value
0,wbc,EMR,CBC,White Blood Cells,×1000/mcL,4~10
1,hb,EMR,CBC,Hemoglobin,g/dL,13~17
2,hct,EMR,CBC,Hematocrit,%,39~52
3,plt,EMR,CBC,Platelet Count,×1000/mcL,130~400
4,esr,EMR,CBC,Erythrocyte Sedimentation Rate,mm/hr,0~9


In [24]:
#filter labs by each Parameter, replace the parameter with the corresponding description
labs_merged = labs.merge(legend, left_on='name', right_on='Parameter')
labs_merged.drop(columns=['Parameter', 'Category', 'Data Source', 'Unit', 'name'], inplace=True)
labs_merged.rename(columns={'Description': 'Parameter'}, inplace=True)
labs_merged.head()

,caseid,dt,result,Parameter,Reference value
0,p000001,1970-01-07 21:07:50,2.9,Albumin,3.3~5.2
1,p000001,1970-01-05 14:59:35,3.2,Albumin,3.3~5.2
2,p000001,1970-01-01 03:30:14,3.4,Albumin,3.3~5.2
3,p000001,1970-01-02 14:17:35,3.6,Albumin,3.3~5.2
4,p000001,1970-01-05 14:59:35,12.0,Alanine Aminotransferase (ALT),1~40


In [30]:
labs_merged['lab_flag'] = 0
#For each parameter, if the value is outside the reference value range (reference value.split('~')[0] and [1]), set lab_flag to 1
for lab in labs_merged['Parameter'].unique():
    ref = labs_merged[labs_merged['Parameter'] == lab]['Reference value'].values[0].split('~')
    labs_merged.loc[
        (labs_merged['Parameter'] == lab) & 
        ((labs_merged['result'] < float(ref[0])) | (labs_merged['result'] > float(ref[1]))),
        'lab_flag'
    ] = 1
labs_merged.head(20)

,caseid,dt,result,Parameter,Reference value,lab_flag
0,p000001,1970-01-07 21:07:50,2.9,Albumin,3.3~5.2,1
1,p000001,1970-01-05 14:59:35,3.2,Albumin,3.3~5.2,1
2,p000001,1970-01-01 03:30:14,3.4,Albumin,3.3~5.2,0
3,p000001,1970-01-02 14:17:35,3.6,Albumin,3.3~5.2,0
4,p000001,1970-01-05 14:59:35,12.0,Alanine Aminotransferase (ALT),1~40,0
5,p000001,1970-01-02 14:17:35,16.0,Alanine Aminotransferase (ALT),1~40,0
6,p000001,1970-01-07 21:07:50,17.0,Alanine Aminotransferase (ALT),1~40,0
7,p000001,1970-01-01 03:30:14,20.0,Alanine Aminotransferase (ALT),1~40,0
8,p000001,1970-01-01 03:30:11,28.0,Activated partial thromboplastin time,26.7~36.6,0
9,p000001,1970-01-05 14:59:35,16.0,Asparate Aminotransferase (AST),1~40,0


In [31]:
#Drop the reference value column
labs_merged.drop(columns=['Reference value'], inplace=True)
labs_merged.to_csv('D:/simedy/CSVs/vital_labs_processed.csv', index=False)